# Working With Dates in Pandas

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Datetime Types

In [ ]:
pd.to_datetime('Jan 1 1970')

In [ ]:
# pd.to_datetime('Jan:1:1970')

In [ ]:
pd.to_datetime('Jan:1:1970', format='%b:%d:%Y')

In [ ]:
url = "https://gist.githubusercontent.com/ryanorsinger/b309f8db19e0ca71b213d4877d835e77/raw/f5841017310e2f4ca070b313529ceec2375336ba/coffee_consumption.csv"
df = pd.read_csv(url)
df.head()

In [ ]:
df.date = pd.to_datetime(df.date)
df.dtypes

## `.dt` accessor

In [ ]:
df.date.dt.day_name()

How many observations are there for each month? Each weekday?

## DateTime Indexes

In [ ]:
df = df.set_index('date').sort_index()
df

In [ ]:
# earliest + latest dates

In [ ]:
# Month of March

In [ ]:
# Subset of march

### Resampling

Upsampling

In [ ]:
by_day = df.asfreq('D')
by_day

In [ ]:
by_day.assign(
    ffill=lambda df: df.coffee_consumption.ffill(),
    bfill=lambda df: df.coffee_consumption.bfill()
).head(15)

In [ ]:
df = df.fillna(0)

Downsampling

In [ ]:
df.resample('3W').mean()

In [ ]:
df.resample('M').sum()

### Rolling Windows

In [ ]:
# 3 day moving average

In [ ]:
# Moving monthly sum

### Lagging and Lead

In [ ]:
pd.concat([
    df.coffee_consumption,
    df.coffee_consumption.diff(),
    df.coffee_consumption.shift(),
], axis=1)

In [ ]:
# different time periods

## Strftime

In [ ]:
df.index.strftime('%B %d, %Y')[:4]

## Timezones

In [ ]:
np.random.seed(123)

idx = pd.date_range('now', freq='H', periods=100)
hourly_df = pd.DataFrame({'x': np.random.choice(list('abc'), 100), 'y': np.random.rand(100)}, index=idx)

In [ ]:
hourly_df.index.tz is None

In [ ]:
hourly_df.tz_localize('America/Chicago')

In [ ]:
hourly_df.tz_localize(None)

In [ ]:
hourly_df = hourly_df.tz_localize('America/New_York')
hourly_df

In [ ]:
hourly_df.tz_convert('America/Los_Angeles')

[Wikipedia: List of Time Zones](https://en.wikipedia.org/wiki/List_of_tz_database_time_zones)

## Timedeltas

In [ ]:
start_date = pd.to_datetime('20210614')
today = pd.to_datetime('20210930')
graduation_date = pd.to_datetime('20211207')

In [ ]:
df['days_since_max'] = (df.index.max() - df.index) // pd.Timedelta('1d')
df.tail(10)

## Plotting

In [ ]:
df.plot()

In [ ]:
df.resample('3M').mean().plot(marker='o')